In [11]:
# !/usr/bin python3
# Fukun Shi, 2019.04.22
# https://chrisostrouchov.com/post/peak_fit_xrd_python/
'''
读取整合peak分析结果
'''
import os
import math
import random

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import optimize, signal

%matplotlib inline

#######################
# 1 输入需要拟合的曲线#

fl = 3e3
fh = 3e6
taul = 1/fh
tauh = 1/fl

maindir = 'D:\\Nutstore\\PhD\\Experiment\\EIS_data_transform\\P1_data\\P1DRT\\Mean\\'
subfile = 'Gaussian'

filepath = maindir + '\\Peaks'                # 读取peaks文件名
fp_center = maindir + 'Peak_center.csv'
fp_amplitude = maindir + 'Peak_amplitude.csv'
fp_sigma = maindir + 'Peak_sigma.csv'
fp_FWHM = maindir + 'Peak_FWHM.csv'
fp_center_freq = maindir + 'Peak_center_freq.csv'

filenames = os.listdir(filepath)
print(filenames)
col = ['Pre','1 min','2 min','5 min','10 min','20 min','30 min','1 h','3 h','5 h','7 h','10 h','24 h']
center = pd.DataFrame(columns=col)
amplitude = pd.DataFrame(columns=col)
sigma = pd.DataFrame(columns=col)
FWHM = pd.DataFrame(columns=col)
for i in range(len(filenames)):
    filename = filenames[i]
    path = os.path.abspath(filepath)
    info = os.path.join(filepath,filename)
    peak = pd.read_excel(info)
    #filename = filename.strip('.xlsx')
    ##################################
    #计算FWHM
    F = pd.DataFrame(columns=col)
    F['F'] = 2*np.sqrt(2*np.log(2))*peak['sigma']                # 计算对数情况下的半极大宽度
    a = pd.DataFrame()
    b = pd.DataFrame()
    a['ub'] = (2*peak['center']-F['F'])/2
    b['lb'] = (2*peak['center']+F['F'])/2
    FWHM.iloc[:,i] = np.power(np.array([10]),b['lb']) - np.power(np.array([10]),a['ub'])
    ###################################
    center.iloc[:,i] = np.power(np.array([10]),peak['center'])   # 将单位转换回s
    sigma.iloc[:,i] = peak['sigma']#np.power(np.array([10]),-2*np.sqrt(2*np.log(2))*peak['sigma'])#
    amplitude.iloc[:,i] = peak['amplitude']


center = center.T
amplitude = amplitude.T
sigma = sigma.T
FWHM = FWHM.T
center_freq = 1/center

center.to_csv(fp_center)
amplitude.to_csv(fp_amplitude)
sigma.to_csv(fp_sigma)
FWHM.to_csv(fp_FWHM)
center_freq.to_csv(fp_center_freq)
print(
    '\n', 'center: ','\n', center,'\n',
      '\n','amplitude: ','\n', amplitude,'\n',
     '\n', 'sigma: ','\n', sigma,'\n',
     '\n', 'FWHM','\n',FWHM)

['0 pre_Peaks.xlsx', '1 min_Peaks.xlsx', '3 min_Peaks.xlsx', '5 min_Peaks.xlsx', '5_10 min_Peaks.xlsx', '5_20 min_Peaks.xlsx', '5_30 min_Peaks.xlsx', 'i1 h_Peaks.xlsx', 'i3 h_Peaks.xlsx', 'i5 h_Peaks.xlsx', 'i7 h_Peaks.xlsx', 'i810 h_Peaks.xlsx', 'i824 h_Peaks.xlsx']

 center:  
                    0         1
Pre     2.888682e-06  0.000082
1 min   1.507036e-05  0.000085
2 min   1.140550e-05  0.000069
5 min   8.107146e-06  0.000068
10 min  2.825644e-06  0.000047
20 min  2.332097e-06  0.000050
30 min  3.767255e-06  0.000055
1 h     3.345688e-07  0.000009
3 h     2.033120e-06  0.000026
5 h     7.455108e-07  0.000014
7 h     3.671100e-07  0.000004
10 h    2.618399e-06  0.000038
24 h    2.305806e-06  0.000032 
 
 amplitude:  
                  0           1
Pre     270.135013  134.807429
1 min   258.635220  130.551907
2 min   137.945529  144.698626
5 min    73.777727  116.340420
10 min   43.933548  117.017354
20 min   32.389088  101.341499
30 min   29.963506   88.086607
1 h       0.815647 

In [23]:
# !/usr/bin python3
# Fukun Shi, 2019.04.22
# https://chrisostrouchov.com/post/peak_fit_xrd_python/
'''
读取peak分析结果，整合
'''
import os
import math
import random

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import optimize, signal

%matplotlib inline


fl = 3e3
fh = 3e6
taul = 1/fh
tauh = 1/fl

maindir = 'D:\\Nustore\\PhD\\Experiment\\EIS_data_transform\\P1_data\\P1DRT\\Mean\\'
subfile = 'Gaussian'

filepath = maindir + '\\Peaks'                # 读取peaks文件名
fp_center = maindir + 'Peak_center.csv'
fp_amplitude = maindir + 'Peak_amplitude.csv'
fp_sigma = maindir + 'Peak_sigma.csv'
fp_FWHM = maindir + 'Peak_FWHM.csv'
fp_center_freq = maindir + 'Inver_Peak_center_freq.csv'

filenames = os.listdir(filepath)
print(filenames)
col = ['Pre','1 min','2 min','5 min','10 min','20 min','30 min','1 h','3 h','5 h','7 h','10 h','24 h']
center = pd.DataFrame(columns=col)
amplitude = pd.DataFrame(columns=col)
sigma = pd.DataFrame(columns=col)
FWHM = pd.DataFrame(columns=col)
for i in range(len(filenames)):
    filename = filenames[i]
    path = os.path.abspath(filepath)
    info = os.path.join(filepath,filename)
    peak = pd.read_excel(info)
    #filename = filename.strip('.xlsx')
    ##################################
    #计算FWHM
    F = pd.DataFrame(columns=col)
    F['F'] = 2*np.sqrt(2*np.log(2))*peak['sigma']                # 计算对数情况下的半极大宽度
    a = pd.DataFrame()
    b = pd.DataFrame()
    a['ub'] = (2*peak['center']-F['F'])/2
    b['lb'] = (2*peak['center']+F['F'])/2
    FWHM.iloc[:,i] = np.power(np.array([10]),b['lb']) - np.power(np.array([10]),a['ub'])
    ###################################
    center.iloc[:,i] = np.power(np.array([10]),peak['center'])   # 将单位转换回s
    sigma.iloc[:,i] = peak['sigma']#np.power(np.array([10]),-2*np.sqrt(2*np.log(2))*peak['sigma'])#
    amplitude.iloc[:,i] = peak['amplitude']
    
center = center.T
amplitude = amplitude.T
sigma = sigma.T
FWHM = FWHM.T

'''center.to_csv(fp_center)
amplitude.to_csv(fp_amplitude)
sigma.to_csv(fp_sigma)
FWHM.to_csv(fp_FWHM)'''
print(
    '\n', 'center: ','\n', center,'\n',
      '\n','amplitude: ','\n', amplitude,'\n',
     '\n', 'sigma: ','\n', sigma,'\n',
     '\n', 'FWHM','\n',FWHM)

['0 pre_Peaks.xlsx', '1 min_Peaks.xlsx', '3 min_Peaks.xlsx', '5 min_Peaks.xlsx', '5_10 min_Peaks.xlsx', '5_20 min_Peaks.xlsx', '5_30 min_Peaks.xlsx', 'i1 h_Peaks.xlsx', 'i3 h_Peaks.xlsx', 'i5 h_Peaks.xlsx', 'i7 h_Peaks.xlsx', 'i810 h_Peaks.xlsx', 'i824 h_Peaks.xlsx']

 center:  
                    0         1
Pre     2.888682e-06  0.000082
1 min   1.507036e-05  0.000085
2 min   1.140550e-05  0.000069
5 min   8.107146e-06  0.000068
10 min  2.825644e-06  0.000047
20 min  2.332097e-06  0.000050
30 min  3.767255e-06  0.000055
1 h     3.345688e-07  0.000009
3 h     2.033120e-06  0.000026
5 h     7.455108e-07  0.000014
7 h     3.671100e-07  0.000004
10 h    2.618399e-06  0.000038
24 h    2.305806e-06  0.000032 
 
 amplitude:  
                  0           1
Pre     270.135013  134.807429
1 min   258.635220  130.551907
2 min   137.945529  144.698626
5 min    73.777727  116.340420
10 min   43.933548  117.017354
20 min   32.389088  101.341499
30 min   29.963506   88.086607
1 h       0.815647 

In [25]:

center_freq = 1/center
center_freq = center_freq.iloc[::-1]
print(center_freq)
center_freq.to_csv(fp_center_freq)
print(fp_center_freq)

                   0              1
24 h    4.336878e+05   31685.071898
10 h    3.819128e+05   26055.575285
7 h     2.723979e+06  277200.477954
5 h     1.341362e+06   69394.804003
3 h     4.918549e+05   37970.369933
1 h     2.988922e+06  115721.383829
30 min  2.654452e+05   18192.458244
20 min  4.287986e+05   19983.616073
10 min  3.539016e+05   21097.379674
5 min   1.233480e+05   14753.356532
2 min   8.767702e+04   14411.093678
1 min   6.635542e+04   11737.518621
Pre     3.461786e+05   12131.035442
D:\Nustore\PhD\Experiment\EIS_data_transform\P1_data\P1DRT\Mean\Inver_Peak_center_freq.csv
